## 🔐 Fine-tuning BERT для класифікації фішингових URL

У цьому туторіалі я покажу, як **дофайнтюнити модель BERT (110 млн параметрів)** для задачі класифікації URL-адрес на `"Safe"` чи `"Not Safe"` (фішинг). Ми пройдемо повний цикл: від підготовки даних і токенізації до тренування та оцінки моделі.


### 🧠 Що таке fine-tuning?

**Fine-tuning** — це процес, коли ми беремо **велику мовну модель, попередньо натреновану на загальному корпусі**, і донавчаємо її на нашій **конкретній задачі**, змінюючи останні шари. У нашому випадку задачею буде класифікація URL.

BERT попередньо навчений на задачах:

* **Masked Language Modeling (MLM)** — вгадування пропущених слів у реченнях
  Приклад:

  > "The weather is \[MASK] today." → "nice"

  Це дозволяє моделі навчитись **розуміти контекст слів з обох боків** — на відміну від автокомплішн-моделей типу GPT, які дивляться тільки в один бік (ліворуч направо). І тому Bert класно пасує під задачі Named Entity Recognition.

* **Next Sentence Prediction (NSP)** — визначення, чи два речення логічно слідують одне за одним.

  Приклад:

  * Речення A: "He went to the store."
  * Речення B: "He bought some milk." → ✅ "is next"
  * Речення B: "The sun is hot." → ❌ "not next"

Ці дві задачі **одночасно** використовувалися під час pretraining'у BERT.
Тому BERT добре розуміє як **структуру речення**, так і **зв'язки між реченнями** — це робить його потужною базою для задач на кшталт класифікації, QA, NER тощо.

### 🧩 Навіщо fine-tuning, якщо є ChatGPT?

Справді, сьогодні можна просто передати приклади ChatGPT і отримати відповідь. Але fine-tuning BERT — це:

  ✅ **Безкоштовно** (після завантаження моделі)

  ✅ **Локально** — без відправки даних у хмару

  ✅ **Швидко** — inference займає мілісекунди

  ✅ **Простий деплоймент** — легко інтегрувати в backend/API

  ✅ **Контроль над даними та якістю** — ви точно знаєте, як і на чому навчена модель

➡️ **Fine-tuning актуальний**, коли вам потрібно:

* працювати з приватними або чутливими даними,
* створити легку production-модель (без залежності від платних API),
* мати повний контроль над поведінкою моделі.


### 💡 Найкращі практики fine-tuning BERT

* Використовуйте `AutoModelForSequenceClassification` для задач класифікації
* Переконайтесь, що `num_labels` та мітки узгоджені
* Використовуйте `DataCollatorWithPadding` для коректного формування батчів
* Робіть `early stopping` або моніторинг метрик на валідації
* Зберігайте токенізатор та `label2id/id2label` для подальшого використання

---

👉 Готові? Тоді починаємо fine-tuning!
Спершу - встановлюємо бібліотеки.
Це займає певний час.


In [ ]:
!pip install transformers datasets evaluate torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

### Завантаження набору даних
Візьмемо набір з HuggingFace Datasets:

In [ ]:
dataset_dict = load_dataset(
    "shawhin/phishing-site-classification"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/98.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/450 [00:00<?, ? examples/s]

In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})

### Завантажуємо модель і токенізатор

На цьому етапі ми завантажуємо готову модель BERT і токенізатор до неї.

- **`AutoTokenizer.from_pretrained(...)`** автоматично підбирає правильний токенізатор до вибраної моделі.
- **`AutoModelForSequenceClassification.from_pretrained(...)`** завантажує попередньо натреновану BERT-модель, адаптовану спеціально для задачі класифікації (тобто на виході — логіти по класах).

Ми також вказуємо:
- `num_labels=2` — кількість класів у задачі (наприклад, "Safe" і "Not Safe").
- `id2label` та `label2id` — словники відповідностей, щоб модель могла:
  - повертати назви класів при інференсі,
  - правильно інтерпретувати числові мітки при тренуванні.

Це важливо для зручного логування, метрик і фінального аналізу.


In [ ]:
model_path = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_path)

id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Це повідомлення означає:

> **Деякі шари моделі `BertForSequenceClassification` були створені з нуля, а не завантажені з чекпойнту `bert-base-uncased`.**
> Зокрема, це шари `classifier.weight` і `classifier.bias`.



### 🔍 Чому так сталося?

* `bert-base-uncased` — це **модель для мовного моделювання**, а не класифікації.
* Але ви ініціалізуєте `BertForSequenceClassification`, яка має **додатковий класифікаційний шар (`classifier`)** для задач типу "текст → клас".
* Цей шар не існує в оригінальному чекпойнті, тому Hugging Face створює його **з нуля**.

💡 **Це нормально і очікувано** для fine-tuning. Ви берете BERT як основу, додаєте класифікатор — і донавчаєте модель під свою задачу.


### ❓Чи можна використовувати `bert-base-uncased` "як є" для класифікації?

**Ні, не напряму.**
Вона не навчена класифікувати речення на `"Safe"` чи `"Not Safe"`.
Ви можете:

1. **Fine-tune** її на своєму датасеті (стандартний шлях).
2. **Або** використовувати zero-shot класифікацію через `pipeline()` з моделлю типу:

   ```python
   from transformers import pipeline

   classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
   result = classifier("your input", candidate_labels=["Safe", "Not Safe"])
   print(result)
   ```

   Але це вже **не BERT**, а модель, навчена спеціально для zero-shot задач (наприклад, BART-MNLI або DeBERTa-MNLI).



In [ ]:
import torch

# model.eval() не допоможе — класифікатор не натренований
model.eval()
inputs = tokenizer("test text", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
print(logits)

tensor([[0.2689, 0.3877]])


Ми отримали **випадкові логіти**, бо `classifier.weight` і `bias` — ще не навчені.


#### Готуємо модель
Нам треба "заморозити" ваги всієї моделі і залишити для файнтюнингу тільки останні кілька шарів.

In [ ]:
# виводимо всі шари
for name, param in model.named_parameters():
   print(name, param.requires_grad)

In [ ]:
# заморожуємо параметри базової моделі
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# і розморожуєм pooling layers - останні, вони і використовуються для класифікації
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [ ]:
for name, param in model.named_parameters():
   print(name, param.requires_grad)

#### Препроцесинг тексту

In [ ]:
# Визначаємо препроцесинг тексту
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
# Токенізуємо весь датасет
tokenized_data = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

`DataCollatorWithPadding` — це клас з Huggingface Transformers, який автоматично додає паддінг до батчів різної довжини під час тренування моделі.

🔹 **Навіщо потрібен?**
Тексти можуть мати різну довжину. Щоб зібрати їх у батч, треба вирівняти їх до однакової довжини — саме це й робить `DataCollatorWithPadding`.

Нам треба використати `DataCollatorWithPadding` на токенізованих даних.

Після цього можна передати `data_collator` у `Trainer`. Він сам подбає про паддінг кожного батчу.


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#### Evaluation

In [ ]:
# Завантажуємо метрики
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # Отримуємо передбачення та справжні мітки
    predictions, labels = eval_pred

    # Застосовуємо softmax, щоб отримати ймовірності
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)

    # Беремо ймовірності позитивного класу для обчислення AUC
    positive_class_probs = probabilities[:, 1]

    # Обчислюємо AUC
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'], 3)

    # Отримуємо передбачені класи (найбільш імовірні)
    predicted_classes = np.argmax(predictions, axis=1)

    # Обчислюємо accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'], 3)

    return {"Accuracy": acc, "AUC": auc}

#### Train model

In [ ]:
# hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-phishing-classifier",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-19-25805d4bc91c>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hpylieva (proxet-ml) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.503300,0.383813,0.818000,0.912000
2,0.408500,0.340465,0.836000,0.930000
3,0.355800,0.314288,0.853000,0.939000
4,0.357100,0.349521,0.851000,0.946000
5,0.349800,0.342193,0.860000,0.948000
6,0.347900,0.292468,0.871000,0.951000
7,0.335500,0.288650,0.880000,0.950000
8,0.312700,0.288252,0.871000,0.950000
9,0.314900,0.284442,0.867000,0.951000
10,0.315000,0.289376,0.867000,0.951000


TrainOutput(global_step=2630, training_loss=0.3600541669606256, metrics={'train_runtime': 299.3734, 'train_samples_per_second': 70.147, 'train_steps_per_second': 8.785, 'total_flos': 706603239165360.0, 'train_loss': 0.3600541669606256, 'epoch': 10.0})

### Використовуємо модель на валідаційному наборі і оцінюємо якість

In [ ]:
# Генеруємо передбачення
predictions = trainer.predict(tokenized_data["validation"])

# Витягаємо логіти та лейбли
logits = predictions.predictions
labels = predictions.label_ids

# Використовуємо нашу написану раніше функцію compute_metrics
metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': np.float64(0.891), 'AUC': np.float64(0.945)}


### Push моделі на hub
Спершу маємо згенерувати токен на `write` на https://huggingface.co/settings/tokens та залогінитись до хабу з цим токеном.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# робимо push моделі на хаб
trainer.push_to_hub()

events.out.tfevents.1747665366.0ff72a670be4.6632.0:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hpylieva/bert-phishing-classifier/commit/a8de35a2c53e3eb73994c87c1744992f6dcfae1f', commit_message='End of training', commit_description='', oid='a8de35a2c53e3eb73994c87c1744992f6dcfae1f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hpylieva/bert-phishing-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='hpylieva/bert-phishing-classifier'), pr_revision=None, pr_num=None)

### Inference на нових екземплярах

In [ ]:
# Спершу перевіряємо, чи доступна CUDA (тобто GPU)
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Використовується GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA недоступна. Використовується CPU.")

# Переносимо модель на відповідний пристрій (GPU або CPU)
model = model.to(device)


In [ ]:
# Токенізуємо вхідний текст
input_text = "https://www.google.com"
input_text = "000mclogin.micloud-object-storage-xc-cos-static-web-hosting-qny.s3.us-east.cloud-object-storage.appdomain.cloud"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Вимикаємо обчислення градієнтів — ми лише робимо передбачення
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# Отримуємо текстову мітку з числового передбачення
predicted_label = model.config.id2label[predictions.item()]
print(f"Передбачена мітка: {predicted_label}")

Передбачена мітка: Safe
